In [54]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time

In [28]:
%store -r df

In [29]:
poke_link = {}

In [30]:
#get dictionary of pokemon and its bulbapedia link
for i in range(len(df)):
    poke_link[df.iat[i,2]]=('https://bulbapedia.bulbagarden.net/wiki/'+df.iat[i, 2]+'_(Pokémon)')

In [38]:
poke_text = {}

In [45]:
#cutoff = dict(list(poke_link.items())[110:])

In [ ]:
#iterate through dictionary to open each pokemon's bulbapedia page
for i, (key, value) in enumerate(poke_link.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)

    try:
        driver.execute_script(f"location.href='{target_url}';")
        time.sleep(3)
        resp = driver.page_source
        soup = BeautifulSoup(resp)
    except:
        print("invalid")
    driver.close()

    #get first tagline info for each pokemon
    start = soup.select_one(".mw-parser-output > p:nth-child(3)")
    end = soup.find("div", {"id":"toc"})
    
    paragraphs = ""
    current = start
    try:
        while current.next_sibling != end:
            paragraphs+=current.get_text()
            current = current.next_sibling
    except:
        print("invalid")

    #save to dictionary
    poke_text[key] = paragraphs


In [50]:
bulb_df = pd.DataFrame(poke_text.items())

In [51]:
bulb_df.head()

,0,1
0,Bulbasaur,Bulbasaur (Japanese: フシギダネ Fushigidane) is a d...
1,Ivysaur,Ivysaur (Japanese: フシギソウ Fushigisou) is a dual...
2,Venusaur,Venusaur (Japanese: フシギバナ Fushigibana) is a du...
3,Mega Venusaur,
4,Charmander,Charmander (Japanese: ヒトカゲ Hitokage) is a Fire...


In [52]:
#renaming columns
bulb_df.columns=["Pokemon", "Bulba_Description"]

In [53]:
bulb_df.head()

,Pokemon,Bulba_Description
0,Bulbasaur,Bulbasaur (Japanese: フシギダネ Fushigidane) is a d...
1,Ivysaur,Ivysaur (Japanese: フシギソウ Fushigisou) is a dual...
2,Venusaur,Venusaur (Japanese: フシギバナ Fushigibana) is a du...
3,Mega Venusaur,
4,Charmander,Charmander (Japanese: ヒトカゲ Hitokage) is a Fire...


In [55]:
bulb_df.to_csv('bulb_df.csv')

In [22]:
%store bulb_df

Stored 'bulb_df' (DataFrame)


In [57]:
bulb_df.head()

,Pokemon,Bulba_Description
0,Bulbasaur,Bulbasaur (Japanese: フシギダネ Fushigidane) is a d...
1,Ivysaur,Ivysaur (Japanese: フシギソウ Fushigisou) is a dual...
2,Venusaur,Venusaur (Japanese: フシギバナ Fushigibana) is a du...
3,Mega Venusaur,
4,Charmander,Charmander (Japanese: ヒトカゲ Hitokage) is a Fire...


In [103]:
%store -r bulb_df

In [104]:
#checking for missing entries
missing_entries = list(bulb_df[bulb_df.eq("").any(axis=1)]["Pokemon"])

In [ ]:
missing_entries

In [106]:
dummydf = bulb_df

In [ ]:
dummydf.head()

In [107]:
#for pokemon with same pages (Venusaur / Mega Venusaur) fill in with same info
for i in range(len(dummydf)):
    if any(dummydf.loc[i, "Pokemon"] in s for s in missing_entries) and dummydf.loc[i, 'Bulba_Description'] != "":
        fill_name = dummydf.loc[i, "Pokemon"]
        fill_text = dummydf.loc[i, "Bulba_Description"]
        for j in range(len(dummydf)):
            if fill_name in dummydf.loc[j, "Pokemon"] and dummydf.loc[j, 'Bulba_Description'] == "":
                #print(fill_name, dummydf.loc[j, "Pokemon"], fill_text)
                dummydf.loc[bulb_df['Pokemon'] == dummydf.loc[j, "Pokemon"], "Bulba_Description"] = fill_text
    fill_text = ""
        

In [124]:
missing_entries = list(bulb_df[dummydf.eq("").any(axis=1)]["Pokemon"])

In [ ]:
missing_entries

In [111]:
len(missing_entries)

75

In [112]:
%store bulb_df

Stored 'bulb_df' (DataFrame)


In [125]:
missing_dict = {}

In [126]:
#for missing entries that have need to have their bulbapedia website links reformatted (ex: Toxtricity Low Key Form)
for i in range(len(missing_entries)):
    original = missing_entries[i]
    
    if "Mega" in missing_entries[i]:
        missing_entries[i] = missing_entries[i].replace("Mega", "")
    if "Galarian" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Galarian", "")
    if "Forme" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Forme", "")
    if "Form" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Form", "")
    if "Style" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Style", "")
    if "Size" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Size", "")
    if "Mode" in missing_entries[i]:
        missing_entries[i]= missing_entries[i].replace("Mode", "")

    missing_entries[i] = missing_entries[i].strip()

    entry = missing_entries[i]

    if " " in missing_entries[i]:
        missing_entries[i] = entry[:entry.index(" ")]

    cleaned = missing_entries[i]

    missing_dict[original] = 'https://bulbapedia.bulbagarden.net/wiki/' + cleaned

In [ ]:
missing_dict

In [143]:
for i, (key, value) in enumerate(missing_dict.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    resp = driver.page_source
    soup = BeautifulSoup(resp)

    driver.close()

    start = soup.select_one(".mw-parser-output > p:nth-child(3)")
    end = soup.find("div", {"id":"toc"})
    
    paragraphs = ""
    current = start
    try:
        while current.next_sibling != end:
            paragraphs+=current.get_text()
            current = current.next_sibling
    except:
        print("invalid")
        
    bulb_df.loc[bulb_df['Pokemon'] == key, "Bulba_Description"] = paragraphs

0 Chansey https://bulbapedia.bulbagarden.net/wiki/Chansey
invalid
1 Mew https://bulbapedia.bulbagarden.net/wiki/Mew
invalid
2 Ho-oh https://bulbapedia.bulbagarden.net/wiki/Ho-oh
3 Deoxys Normal Forme https://bulbapedia.bulbagarden.net/wiki/Deoxys
4 Deoxys Attack Forme https://bulbapedia.bulbagarden.net/wiki/Deoxys
5 Deoxys Defense Forme https://bulbapedia.bulbagarden.net/wiki/Deoxys
6 Deoxys Speed Forme https://bulbapedia.bulbagarden.net/wiki/Deoxys
7 Wormadam Plant Cloak https://bulbapedia.bulbagarden.net/wiki/Wormadam
8 Wormadam Sandy Cloak https://bulbapedia.bulbagarden.net/wiki/Wormadam
9 Wormadam Trash Cloak https://bulbapedia.bulbagarden.net/wiki/Wormadam
10 Giratina Altered Forme https://bulbapedia.bulbagarden.net/wiki/Giratina
11 Giratina Origin Forme https://bulbapedia.bulbagarden.net/wiki/Giratina
12 Shaymin Land Forme https://bulbapedia.bulbagarden.net/wiki/Shaymin
13 Shaymin Sky Forme https://bulbapedia.bulbagarden.net/wiki/Shaymin
14 Basculin Red-Striped Form https://bulba

In [144]:
missing_entries = list(bulb_df[bulb_df.eq("").any(axis=1)]["Pokemon"])

In [145]:
#since so few missing entries, manually just grabbing the last few
missing_entries

['Chansey', 'Mew', 'Kartana', 'Drizzile', 'Polteageist']

In [146]:
chans_str = "Chansey (Japanese: ラッキー Lucky) is a Normal-type Pokémon introduced in Generation I.\n It evolves from Happiny when leveled up while holding an Oval Stone during the day and evolves into Blissey when leveled up with high friendship. "

In [147]:
mew_str = "Mew (Japanese: ミュウ Mew) is a Psychic-type Mythical Pokémon introduced in Generation I.\n It is not known to evolve into or from any other Pokémon.\n Mew is notable for its unique ability to learn every Technical Machine, Technical Record, Hidden Machine, and Move Tutor move (except those exclusive to a particular Pokémon or group of Pokémon, such as Volt Tackle and Secret Sword).\n It is a member of the Mew duo along with Mewtwo."

In [148]:
kartana_str = "Kartana (Japanese: カミツルギ Kamiturugi) is a dual-type Grass/Steel Ultra Beast introduced in Generation VII.\n It is not known to evolve into or from any other Pokémon.\n It is code named UB-04 Blade (Japanese: UB04 Slash)."


In [149]:
driz_str="Drizzile (Japanese: ジメレオン Jimereon) is a Water-type Pokémon introduced in Generation VIII.\n It evolves from Sobble starting at level 16 and evolves into Inteleon starting at level 35. "


In [150]:
poltea_str = "Polteageist (Japanese: ポットデス Potdeath) is a Ghost-type Pokémon introduced in Generation VIII.\n It evolves from Sinistea when exposed to a Cracked Pot or a Chipped Pot, depending on Sinistea's form. Polteageist's form will be the same as it was as a Sinistea.\n The Phony Form[1][2] (if it lacks a stamp of authenticity) requires a Cracked Pot to evolve.\n The Antique Form[1][3] (if it has a stamp of authenticity) requires a Chipped Pot to evolve.\n\n It is ecologically similar to Sinistcha; however, the two are separate species. "


In [151]:
bulb_df.loc[bulb_df['Pokemon'] == "Chansey", "Bulba_Description"] = chans_str
bulb_df.loc[bulb_df['Pokemon'] == 'Mew', "Bulba_Description"] = mew_str
bulb_df.loc[bulb_df['Pokemon'] == 'Kartana', "Bulba_Description"] = kartana_str
bulb_df.loc[bulb_df['Pokemon'] == 'Drizzile', "Bulba_Description"] =driz_str
bulb_df.loc[bulb_df['Pokemon'] == 'Polteageist', "Bulba_Description"] =poltea_str

In [152]:
%store bulb_df

Stored 'bulb_df' (DataFrame)
